# Прогноз дохода клиентов

 - Задача прогнозирования дохода имеет особенное значение в Банке. 
Эта информация помогает точнее и более релевантно подбирать продукты и условия их приобретения, что в свою очередь вносит существенный вклад в прибыль Банка. 
 - Помимо ценности для Банка, оценка дохода является регуляторным требованием ЦБ в части расчета предельно допустимой кредитной нагрузки для клиента (далее ПДН)

# 1) Импорт библиотек

In [12]:
import pandas as pd # Библиотека для работы с данными в формате CSV
import seaborn as sns # Библиотека для визуализации данных
import matplotlib.pyplot as plt # Библиотека для визуализации данных

# 2) Загрузка исторических данных

In [13]:
df_train = pd.read_csv(
    'hackathon_income_train.csv',
    sep=';'
)
df_test = pd.read_csv(
    'hackathon_income_test.csv',
    sep=';'
)

df = pd.concat(
    [df_train, df_test],
    ignore_index=True
    )

In [14]:
df.info()   # Информация об исторических данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Columns: 224 entries, id to first_salary_income
dtypes: float64(34), int64(1), object(189)
memory usage: 256.3+ MB


In [15]:
df.describe()   # Описание исторических данных

,id,hdb_bki_total_max_limit,hdb_bki_total_cc_max_limit,hdb_bki_total_pil_max_limit,hdb_bki_active_cc_max_limit,hdb_bki_other_active_pil_outstanding,hdb_bki_total_products,hdb_bki_total_max_overdue_sum,bki_total_auto_cnt,hdb_bki_total_ip_max_limit,...,hdb_bki_total_micro_cnt,hdb_bki_active_pil_cnt,hdb_bki_total_cc_max_overdue,hdb_bki_total_pil_last_days,hdb_bki_active_pil_max_limit,hdb_bki_total_pil_cnt,hdb_bki_total_pil_max_del90,hdb_bki_last_product_days,hdb_bki_other_active_ip_outstanding,hdb_bki_active_cc_max_overdue
count,150000.000000,1.302220e+05,1.209040e+05,1.176430e+05,1.083320e+05,6.597800e+04,137285.000000,1.297470e+05,125765.000000,4.032400e+04,...,130222.000000,130222.000000,1.200890e+05,117643.000000,7.724500e+04,130222.000000,110577.000000,130222.000000,2.762900e+04,1.059410e+05
mean,74999.500000,1.361677e+06,2.502501e+05,5.955726e+05,2.240513e+05,5.017570e+05,13.369938,5.848935e+04,0.203681,3.166614e+06,...,2.821321,1.568084,1.623517e+04,724.035268,5.877321e+05,6.782441,7.489116,495.482614,3.126427e+06,6.978554e+03
std,43301.414527,2.549925e+06,3.421225e+05,9.246573e+05,2.474795e+05,9.556245e+05,21.045049,4.770585e+05,0.784461,3.080195e+06,...,9.633014,2.273131,7.156926e+04,964.637166,8.733440e+05,7.201920,33.160232,724.150890,3.962851e+06,3.498027e+04
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,1.200000e+04,...,0.000000,0.000000,0.000000e+00,2.000000,0.000000e+00,0.000000,0.000000,2.000000,0.000000e+00,0.000000e+00
25%,37499.750000,1.810000e+05,7.000000e+04,1.141170e+05,7.000000e+04,3.896693e+04,4.000000,0.000000e+00,0.000000,1.430000e+06,...,0.000000,0.000000,0.000000e+00,153.000000,1.175000e+05,2.000000,0.000000,101.000000,9.802998e+05,0.000000e+00
50%,74999.500000,5.622220e+05,1.550000e+05,3.285690e+05,1.500000e+05,1.965894e+05,8.000000,4.535000e+03,0.000000,2.352656e+06,...,0.000000,1.000000,1.500000e+02,350.000000,3.290000e+05,5.000000,0.000000,237.000000,2.082831e+06,0.000000e+00
75%,112499.250000,1.680000e+06,3.000000e+05,7.224026e+05,2.950000e+05,5.835298e+05,16.000000,2.272650e+04,0.000000,3.961857e+06,...,1.000000,2.000000,6.374000e+03,899.000000,7.210000e+05,10.000000,0.000000,559.000000,3.976280e+06,1.813000e+03
max,149999.000000,4.000000e+08,1.500000e+07,6.200000e+07,7.000000e+06,2.724127e+07,993.000000,1.306604e+08,24.000000,1.759704e+08,...,354.000000,57.000000,6.808735e+06,7239.000000,5.250000e+07,122.000000,1183.000000,7254.000000,1.783505e+08,1.519780e+06


In [16]:
df.isnull().sum()   # Количество пропусков

id                                                   0
dt                                                   0
target                                           73214
turn_cur_cr_avg_act_v2                           33678
salary_6to12m_avg                               127491
                                                 ...  
label_Above_1M_share_r1                          82335
transaction_category_supermarket_sum_cnt_d15     80649
max_balance_rur_amt_1m_af                       126877
w                                                73214
first_salary_income                             141332
Length: 224, dtype: int64

In [17]:
df.duplicated().sum() # Количество дубликатов

np.int64(0)

In [18]:
df.nunique()    # Количество уникальных значений

id                                              150000
dt                                                  11
target                                           73426
turn_cur_cr_avg_act_v2                          109382
salary_6to12m_avg                                22507
                                                 ...  
label_Above_1M_share_r1                           1308
transaction_category_supermarket_sum_cnt_d15       108
max_balance_rur_amt_1m_af                         8116
w                                                70630
first_salary_income                               8668
Length: 224, dtype: int64

# 3) Разведочный статистический анализ

Гипотеза №1. Целевая переменная обладает несимметричным распределением величины, вследствие чего придется использовать логарифмирование.

Гипотеза №2. Возраст влияет на доход клиента. Юные и престарелые клиенты, скорее всего, выделяются низкими доходами. В то врем как клиенты среднего возраста способны обладать высокими доходами.

Гипотеза №3. Чем выше усредненная ЗП клиента, тем выше его доход, поскольку обычно ЗП является существенной частью дохода.

Гипотеза №4. Доход клиента зависит от среднего по региону, в котором он проживает, пос